# Segmentation of Kinect Data

In [1]:
#Moving average smoothing
def filter_column(column, window_size):
    return column.rolling(window=window_size).mean()


In [16]:
#导入所用包
import pandas as pd
import numpy as np
import os
from datetime import datetime


def kinect_preprocess(folder_path):
    # #Getting all files and folders in a specified directory
    contents = os.listdir(folder_path)
    timestamp_path="\\Data_new\\Raw\\Timestamp"
    
    #Traverse the folder 'sample' under the 'IMU' folder and the 'Timestamp' folder
    for item in contents[:]:
        item_path = os.path.join(folder_path, item)
        #Browse the timestamp folder for the corresponding item
        item_timestamp= os.path.join(timestamp_path,item)
        print('folder：' + item_timestamp)

        if os.path.isdir(item_path):
            print('folder：' + item_path)
        
        
        item_contents = os.listdir(item_path)
        #Traverse the subfolders 'time01' and 'time02' under the 'sample' folder
        for t in item_contents[:]:
            item_path_time = os.path.join(item_path, t) 
            item_timestamp_time= os.path.join(item_timestamp, t) 
            print('folder：'+item_path_time)
            print('folder：'+item_timestamp_time)
            
            
            #Browse the 'timeID' file under the 'sample' folder
            timestamp=pd.read_csv(item_timestamp_time+'.csv')
            
            
            file_contents = os.listdir(item_path_time)
            
            #Traverse the 'task' files under the 'time' folder
            for file in file_contents[:4]:
                if file[:-4] == 'task_normal':
                    row = 0
                if file[:-4] == 'task_cup':
                    row = 1
                if file[:-4] == 'task_text':
                    row = 2
                if file[:-4] == 'task_web':
                    row = 3

                file_path = os.path.join(item_path_time, file)
                print('file：'+file_path)
                
                #Import files
                ki=pd.read_csv(file_path)
                ki1=ki.copy()
                
                ki1['timestamp2']=0
                import time
                #Convert timestamps to numerical format
                for i in range(0,len(ki1)):
                    time_num = ki1.iloc[i,0][:-8]
                    time_array = time.strptime(time_num, "%Y-%m-%d %H:%M:%S")
                    time_stamp = time.mktime(time_array)
                    time_stamp =time_stamp+float(ki1.iloc[i,0][-8:-4])
                    ki1.iloc[i,26]=time_stamp
    
                #Segment different nodes of timestamp records
                start_kinect=timestamp.iloc[row,2][:-7]
                start_turn=timestamp.iloc[row,3][:-7]
                end_turn=timestamp.iloc[row,4][:-7]
                end_kinect=timestamp.iloc[row,5][:-7]
                
                #Align timestamps with Kinect timestamps
                import time
                time_array_start_kinect = time.strptime(start_kinect, "%Y-%m-%d-%H:%M:%S")
                time_array_start_turn = time.strptime(start_turn, "%Y-%m-%d-%H:%M:%S")
                time_array_end_turn = time.strptime(end_turn, "%Y-%m-%d-%H:%M:%S")
                time_array_end_kinect = time.strptime(end_kinect, "%Y-%m-%d-%H:%M:%S")
                
                timestamp_start_kinect = time.mktime(time_array_start_kinect)
                timestamp_start_turn = time.mktime(time_array_start_turn)
                timestamp_end_turn = time.mktime(time_array_end_turn)
                timestamp_end_kinect = time.mktime(time_array_end_kinect)
                
                timestamp_start_kinect= timestamp_start_kinect+float(timestamp.iloc[row,2][-7:-3])
                timestamp_start_turn= timestamp_start_turn+float(timestamp.iloc[row,3][-7:-3])
                timestamp_end_turn= timestamp_end_turn+float(timestamp.iloc[row,4][-7:-3])
                timestamp_end_kinect= timestamp_end_kinect+float(timestamp.iloc[row,5][-7:-3])
                
                
                #go: start of kinect ~ start of turn
                ki_go=ki1.copy()
                for i in range(0,len(ki_go)):
                    timestamp2 = ki_go.iloc[i,26]
                    if timestamp2 >= timestamp_start_kinect :
                        ki_go.drop(ki_go.head(i).index,inplace=True)
                        break
                ki_go.reset_index(drop=True,inplace=True)
                for i in range(0,len(ki_go)):
                    timestamp2 = ki_go.iloc[i,26]
                    if timestamp2 > timestamp_start_turn :
                        
                        
                        break
                
                ki_go=ki_go.drop(columns=['timestamp'])
                ki_go= ki_go[ki_go.columns[-1:].tolist() + ki_go.columns[:-1].tolist()]
                ki_go['timestamp2'] = ki_go['timestamp2'].astype(str)
                
                #Separate (X,Y,Z) into individual components
                ki_go2=ki_go.copy()
                ki_go2 = ki_go2[ki_go2['keypoint_0'] != 'None_tracked']
                ki_go2.reset_index(drop=True,inplace=True)
                for j in range(1,26):
                    ti=j-1  
                    ki_go2[['X'+str(ti),'Y'+str(ti),'Z'+str(ti)]]=ki_go2.iloc[:,j].str.split(expand=True)
                ki_go2=ki_go2.drop(ki_go2.columns[1:26], axis=1)
                #Remove rows where the value is 0
                zero_rows_go = (ki_go2 == '0.0').any(axis=1)
                ki_go2 = ki_go2[~zero_rows_go]
                #Perform smoothing
                window_size = 3
             
                ki_go2.iloc[:,1:].apply(lambda x: filter_column(x, window_size))
                
                #重新排序,改变Y轴和Z轴
                ki_go2 = ki_go2.apply(pd.to_numeric, errors='coerce')
                new_order = ['timestamp2','X0', 'Z0', 'Y0','X1', 'Z1', 'Y1','X2', 'Z2', 'Y2','X3', 'Z3', 'Y3','X4', 'Z4', 'Y4','X5', 'Z5', 'Y5','X6', 'Z6', 'Y6','X7', 'Z7', 'Y7','X8', 'Z8', 'Y8','X9', 'Z9', 'Y9','X10', 'Z10', 'Y10','X11', 'Z11', 'Y11','X12', 'Z12', 'Y12', 'X13', 'Z13', 'Y13','X14', 'Z14', 'Y14','X15', 'Z15', 'Y15','X16', 'Z16', 'Y16','X17', 'Z17', 'Y17','X18', 'Z18', 'Y18','X19', 'Z19', 'Y19', 'X20', 'Z20', 'Y20','X21', 'Z21', 'Y21','X22', 'Z22', 'Y22','X23', 'Z23', 'Y23','X24', 'Z24', 'Y24'
                ]
                ki_go3 = ki_go2[new_order]
            
                #重新排号，修改列标题
                # 定义新的列名列表,X轴反方向
                new_columns = ['Timestamp','X0', 'Y0', 'Z0','X1', 'Y1', 'Z1','X2', 'Y2', 'Z2','X3', 'Y3', 'Z3','X4', 'Y4', 'Z4','X5', 'Y5', 'Z5','X6', 'Y6', 'Z6','X7', 'Y7', 'Z7','X8', 'Y8', 'Z8','X9', 'Y9', 'Z9','X10', 'Y10', 'Z10','X11', 'Y11', 'Z11','X12', 'Y12', 'Z12', 'X13', 'Y13', 'Z13','X14', 'Y14', 'Z14','X15', 'Y15', 'Z15','X16', 'Y16', 'Z16','X17', 'Y17', 'Z17','X18', 'Y18', 'Z18','X19', 'Y19', 'Z19', 'X20', 'Y20', 'Z20','X21', 'Y21', 'Z21','X22', 'Y22', 'Z22','X23', 'Y23', 'Z23','X24', 'Y24', 'Z24'
                ]
                # 直接赋值给 df.columns
                ki_go3.columns = new_columns
                
                # 列出你想要反转的列
                columns_to_reverse = ['X0','X1', 'X2','X3','X4','X5','X6','X7','X8','X9','X10','X11','X12','X13','X14','X15','X16','X17','X18','X19','X20','X21','X22','X23','X24']
                
                # 取反
                ki_go3[columns_to_reverse] = -ki_go3[columns_to_reverse]
                
                
                
              
                #turn: start of turn ~ end of turn
                ki_turn=ki1.copy()
                for i in range(0,len(ki_turn)):
                    timestamp2 = ki_turn.iloc[i,26]
                    if timestamp2 >= timestamp_start_turn :
                        ki_turn.drop(ki_turn.head(i).index,inplace=True)
                        break
                ki_turn.reset_index(drop=True,inplace=True)
                for i in range(0,len(ki_turn)):
                    timestamp2 = ki_turn.iloc[i,26]
                    if timestamp2 > timestamp_end_turn :
                        ki_turn.drop(ki_turn.tail(len(ki_turn)-i).index,inplace=True)
                        break
                
                ki_turn=ki_turn.drop(columns=['timestamp'])
                ki_turn= ki_turn[ki_turn.columns[-1:].tolist() + ki_turn.columns[:-1].tolist()]
                ki_turn['timestamp2'] =ki_turn['timestamp2'].astype(str)
                
                ki_turn2=ki_turn.copy()
                ki_turn2 = ki_turn2[ki_turn2['keypoint_0'] != 'None_tracked']
                ki_turn2.reset_index(drop=True,inplace=True)
                for j in range(1,26):
                    ti=j-1  
                    ki_turn2[['X'+str(ti),'Y'+str(ti),'Z'+str(ti)]]=ki_turn2.iloc[:,j].str.split(expand=True)
                ki_turn2=ki_turn2.drop(ki_turn2.columns[1:26], axis=1)  
               
                zero_rows_turn =(ki_turn2 == '0.0').any(axis=1)
                ki_turn2 = ki_turn2[~zero_rows_turn]
            
                window_size = 3
       
                ki_turn2.iloc[:,1:].apply(lambda x: filter_column(x, window_size))
                
                #重新排序,改变Y轴和Z轴
                ki_turn2 = ki_turn2.apply(pd.to_numeric, errors='coerce')
                new_order = ['timestamp2','X0', 'Z0', 'Y0','X1', 'Z1', 'Y1','X2', 'Z2', 'Y2','X3', 'Z3', 'Y3','X4', 'Z4', 'Y4','X5', 'Z5', 'Y5','X6', 'Z6', 'Y6','X7', 'Z7', 'Y7','X8', 'Z8', 'Y8','X9', 'Z9', 'Y9','X10', 'Z10', 'Y10','X11', 'Z11', 'Y11','X12', 'Z12', 'Y12', 'X13', 'Z13', 'Y13','X14', 'Z14', 'Y14','X15', 'Z15', 'Y15','X16', 'Z16', 'Y16','X17', 'Z17', 'Y17','X18', 'Z18', 'Y18','X19', 'Z19', 'Y19', 'X20', 'Z20', 'Y20','X21', 'Z21', 'Y21','X22', 'Z22', 'Y22','X23', 'Z23', 'Y23','X24', 'Z24', 'Y24'
                ]
                ki_turn3 = ki_turn2[new_order]
            
                #重新排号，修改列标题
                # 定义新的列名列表,X轴反方向
                new_columns = ['Timestamp','X0', 'Y0', 'Z0','X1', 'Y1', 'Z1','X2', 'Y2', 'Z2','X3', 'Y3', 'Z3','X4', 'Y4', 'Z4','X5', 'Y5', 'Z5','X6', 'Y6', 'Z6','X7', 'Y7', 'Z7','X8', 'Y8', 'Z8','X9', 'Y9', 'Z9','X10', 'Y10', 'Z10','X11', 'Y11', 'Z11','X12', 'Y12', 'Z12', 'X13', 'Y13', 'Z13','X14', 'Y14', 'Z14','X15', 'Y15', 'Z15','X16', 'Y16', 'Z16','X17', 'Y17', 'Z17','X18', 'Y18', 'Z18','X19', 'Y19', 'Z19', 'X20', 'Y20', 'Z20','X21', 'Y21', 'Z21','X22', 'Y22', 'Z22','X23', 'Y23', 'Z23','X24','Y24', 'Z24'
                ]
                # 直接赋值给 df.columns
                ki_turn3.columns = new_columns
                # 列出你想要反转的列
                columns_to_reverse = ['X0','X1', 'X2','X3','X4','X5','X6','X7','X8','X9','X10','X11','X12','X13','X14','X15','X16','X17','X18','X19','X20','X21','X22','X23','X24']
                
                # 取反
                ki_turn3[columns_to_reverse] = -ki_turn3[columns_to_reverse]
                
                
                
                
                #back: end of turn ~ end of kinect
                ki_back=ki1.copy()
                for i in range(0,len(ki_back)):
                    timestamp2 = ki_back.iloc[i,26]
                    if timestamp2 >= timestamp_end_turn :
                        ki_back.drop(ki_back.head(i).index,inplace=True)
                        break
                ki_back.reset_index(drop=True,inplace=True)
                for i in range(0,len(ki_back)):
                    timestamp2 = ki_back.iloc[i,26]
                    if timestamp2 > timestamp_end_kinect :
                        ki_back.drop(ki_back.tail(len(ki_back)-i).index,inplace=True)
                        break
                
                ki_back=ki_back.drop(columns=['timestamp'])
                ki_back= ki_back[ki_back.columns[-1:].tolist() + ki_back.columns[:-1].tolist()]
                ki_back['timestamp2'] =ki_back['timestamp2'].astype(str)
                
                ki_back2=ki_back.copy()
                ki_back2 = ki_back2[ki_back2['keypoint_0'] != 'None_tracked']
                ki_back2.reset_index(drop=True,inplace=True)
                for j in range(1,26):
                    ti=j-1  
                    ki_back2[['X'+str(ti),'Y'+str(ti),'Z'+str(ti)]]=ki_back2.iloc[:,j].str.split(expand=True)
                ki_back2=ki_back2.drop(ki_back2.columns[1:26], axis=1)      
              
                zero_rows_back =(ki_back2 == '0.0').any(axis=1)
                ki_back2 = ki_back2[~zero_rows_back]
                
                #Swap the coordinates of the left and right points
                cols = list(ki_back2.columns)
                cols[13], cols[14], cols[15], cols[16], cols[17], cols[18],cols[19],cols[20],cols[21],cols[22],cols[23],cols[24],cols[25],cols[26],cols[27],cols[28],cols[29],cols[30],cols[31],cols[32],cols[33],cols[34],cols[35],cols[36],cols[37],cols[38],cols[39],cols[40],cols[41],cols[42],cols[43],cols[44],cols[45],cols[46],cols[47],cols[48],cols[49],cols[50],cols[51],cols[52],cols[53],cols[54],cols[55],cols[56],cols[57],cols[58],cols[59],cols[60],cols[64],cols[65],cols[66],cols[67],cols[68],cols[69],cols[70],cols[71],cols[72],cols[73],cols[74],cols[75] = cols[25],cols[26],cols[27],cols[28],cols[29],cols[30],cols[31],cols[32],cols[33],cols[34],cols[35],cols[36],cols[13], cols[14], cols[15], cols[16], cols[17], cols[18],cols[19],cols[20],cols[21],cols[22],cols[23],cols[24],cols[49],cols[50],cols[51],cols[52],cols[53],cols[54],cols[55],cols[56],cols[57],cols[58],cols[59],cols[60],cols[37],cols[38],cols[39],cols[40],cols[41],cols[42],cols[43],cols[44],cols[45],cols[46],cols[47],cols[48],cols[70],cols[71],cols[72],cols[73],cols[74],cols[75],cols[64],cols[65],cols[66],cols[67],cols[68],cols[69]
                ki_back2=ki_back2[cols]
                
                ki_back2.columns = ['timestamp2','X0','Y0','Z0','X1',	'Y1',	'Z1',	'X2',	'Y2',	'Z2',	'X3',	'Y3',	'Z3',	'X4',	'Y4',	'Z4',	'X5',	'Y5',	'Z5',	'X6',	'Y6',	'Z6',	'X7'	,'Y7',	'Z7',	'X8',	'Y8',	'Z8',	'X9'	,'Y9',	'Z9',	'X10',	'Y10',	'Z10',	'X11',	'Y11'	,'Z11'	,'X12'	,'Y12',	'Z12',	'X13',	'Y13',	'Z13',	'X14',	'Y14',	'Z14',	'X15'	,'Y15',	'Z15',	'X16',	'Y16',	'Z16',	'X17'	,'Y17',	'Z17'	,'X18'	,'Y18'	,'Z18',	'X19',	'Y19',	'Z19',	'X20',	'Y20',	'Z20',	'X21',	'Y21',	'Z21',	'X22',	'Y22','Z22',	'X23',	'Y23',	'Z23',	'X24',	'Y24',	'Z24']

                window_size = 3
    
                ki_back2.iloc[:,1:].apply(lambda x: filter_column(x, window_size))
                ki_back2 = ki_back2.apply(pd.to_numeric, errors='coerce')
                
                #重新排序,改变Y轴和Z轴
                
                new_order = ['timestamp2','X0', 'Z0', 'Y0','X1', 'Z1', 'Y1','X2', 'Z2', 'Y2','X3', 'Z3', 'Y3','X4', 'Z4', 'Y4','X5', 'Z5', 'Y5','X6', 'Z6', 'Y6','X7', 'Z7', 'Y7','X8', 'Z8', 'Y8','X9', 'Z9', 'Y9','X10', 'Z10', 'Y10','X11', 'Z11', 'Y11','X12', 'Z12', 'Y12', 'X13', 'Z13', 'Y13','X14', 'Z14', 'Y14','X15', 'Z15', 'Y15','X16', 'Z16', 'Y16','X17', 'Z17', 'Y17','X18', 'Z18', 'Y18','X19', 'Z19', 'Y19', 'X20', 'Z20', 'Y20','X21', 'Z21', 'Y21','X22', 'Z22', 'Y22','X23', 'Z23', 'Y23','X24', 'Z24', 'Y24'
                ]
                ki_back3 = ki_back2[new_order]
            
                #重新排号，修改列标题
                # 定义新的列名列表,X轴反方向
                new_columns = ['Timestamp','X0', 'Y0', 'Z0','X1', 'Y1', 'Z1','X2', 'Y2', 'Z2','X3', 'Y3', 'Z3','X4', 'Y4', 'Z4','X5', 'Y5', 'Z5','X6', 'Y6', 'Z6','X7', 'Y7', 'Z7','X8', 'Y8', 'Z8','X9', 'Y9', 'Z9','X10', 'Y10', 'Z10','X11', 'Y11', 'Z11','X12', 'Y12', 'Z12', 'X13', 'Y13', 'Z13','X14', 'Y14', 'Z14','X15', 'Y15', 'Z15','X16', 'Y16', 'Z16','X17', 'Y17', 'Z17','X18', 'Y18', 'Z18','X19', 'Y19', 'Z19', 'X20', 'Y20', 'Z20','X21', 'Y21', 'Z21','X22', 'Y22', 'Z22','X23', 'Y23', 'Z23','X24', 'Y24', 'Z24'
                ]
                # 直接赋值给 df.columns
                ki_back3.columns = new_columns
                # 列出你想要反转的列
                columns_to_reverse = ['X0','X1', 'X2','X3','X4','X5','X6','X7','X8','X9','X10','X11','X12','X13','X14','X15','X16','X17','X18','X19','X20','X21','X22','X23','X24']
                
                # 取反
                ki_back3[columns_to_reverse] = -ki_back3[columns_to_reverse]
                
                
                
                #三个阶段融合
                ki3 = pd.concat([ki_go3, ki_turn3, ki_back3], ignore_index=True)

                ki3 = ki3.apply(pd.to_numeric, errors='coerce')
                
                
                # 坐标系一致
                
                
                
                
                
                
                
                ki_go3.to_csv("\\Data_new\\Processed\\Kinect_2m\\"+t+'\\go\\'+file[:-4]+'\\'+item+'_'+file[:-4]+'.csv',index=False)
                ki_turn3.to_csv("\\Data_new\\Processed\\Kinect_2m\\"+t+'\\turn\\'+file[:-4]+'\\'+item+'_'+file[:-4]+'.csv',index=False)
                ki_back3.to_csv("\\Data_new\\Processed\\Kinect_2m\\"+t+'\\back\\'+file[:-4]+'\\'+item+'_'+file[:-4]+'.csv',index=False)

In [17]:
kinect_preprocess("\\Data_new\\Raw\\Kinect")

folder：D:\A-lyy\论文\多元传感在步态分析中的应用\论文\提交数据集和代码\Data_new\Raw\Timestamp\sample01
folder：D:\A-lyy\论文\多元传感在步态分析中的应用\论文\提交数据集和代码\Data_new\Raw\Kinect\sample01
folder：D:\A-lyy\论文\多元传感在步态分析中的应用\论文\提交数据集和代码\Data_new\Raw\Kinect\sample01\time01
folder：D:\A-lyy\论文\多元传感在步态分析中的应用\论文\提交数据集和代码\Data_new\Raw\Timestamp\sample01\time01
file：D:\A-lyy\论文\多元传感在步态分析中的应用\论文\提交数据集和代码\Data_new\Raw\Kinect\sample01\time01\task_cup.csv


TypeError: '>=' not supported between instances of 'str' and 'float'